In [4]:
!apt-get install build-essential libpoppler-cpp-dev pkg-config python-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package python-dev is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  python2-dev python2 python-dev-is-python3

E: Package 'python-dev' has no installation candidate


In [12]:
!pip install pdftotext

  Using cached pdftotext-2.2.2.tar.gz (113 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pdftotext
  Running setup.py clean for pdftotext
Failed to build pdftotext
ERROR: Could not build wheels for pdftotext, which is required to install pyproject.toml-based projects


In [5]:
!pip install tabula==1.3.1

  Using cached tabula_py-1.3.1-py3-none-any.whl (10.4 MB)


In [10]:
!pip install pyPDF4

  Using cached PyPDF4-1.27.0-py3-none-any.whl


In [7]:
import sys
import re
import os
import json
import csv
from datetime import datetime
from io import StringIO
import math
import numpy as np
import pandas as pd
import tabula

def empty_narration(amount):
    category = ''
    if amount < 0:
        category = "Transfer out"
    elif amount >= 0:
        category = "Transfer in"

    return category

def test_list(txn_list, txn_id=None):
    for i, d in enumerate(txn_list):
        if i == 0:
            continue
        round_balance = round(txn_list[i-1]['balance'] + d['amount'], 2)
        if round_balance != d['balance']:
            print('Transaction mismatch, ref:')
            print(txn_list[i - 1])
            print(txn_list[i])
            raise('Unable to proceed further')
    print('validated...')

def format_dt_ymd(date_str,bank=''):
    date_value = None
    try:
        date_str = datetime.strptime(date_str, '%d/%m/%Y')
    except ValueError:
        date_str = datetime.strptime(date_str, '%d-%b-%y')
    date_value = str(date_str.year) + '-' + \
        '{:02d}'.format(date_str.month) + '-' + \
        '{:02d}'.format(date_str.day)
    return date_value

def bank_txns(file_path,doc_pwd=None):
    txn_list = []

    txns_table = None

    if doc_pwd:
        txns_tables = tabula.read_pdf(
            file_path, password=doc_pwd, pages='all', spreadsheet=True)
    else:
        txns_tables = tabula.read_pdf(
            file_path, pages='all', spreadsheet=True,multiple_tables=True)

    field_list = {
        'txn_date': '',
        'description': '',
        'ref_no': '',
        'debit': '',
        'credit': '',
        'balance': ''
    }

    for txns_table in txns_tables[1:]:
      new_header=txns_table.iloc[0]
      txns_table.columns=new_header
      field_list['txn_date'] = new_header[0]
      field_list['description'] = new_header[2]
      field_list['debit'] = new_header[5]
      field_list['credit'] = new_header[6]
      field_list['balance'] = new_header[7]

      for ind in range(len(txns_table)):
          if(ind == 0):
            continue
          txn_dict = {}
          date_str = txns_table[field_list['txn_date']][ind]
          narrat = txns_table[field_list['description']][ind]
          if  pd.isna(date_str) or pd.isna(narrat):
            continue
          date_str = date_str.replace('\r', '')
          date_field = format_dt_ymd(date_str)
          txn_dict['date'] = date_field
          txn_dict['narration']=txns_table[field_list['description']][ind]
          txn_dict['narration'] = re.sub(r"\s+", " ", txn_dict['narration'])

          debit, credit = None, None

          debit = txns_table[field_list['debit']][ind]
          credit = txns_table[field_list['credit']][ind]

          if isinstance(debit, str):
              debit = debit.replace(',', '')
              debit = float(debit)

          if isinstance(credit, str):
              credit = credit.replace(',', '')
              credit = float(credit)

          if not np.isnan(debit):
              txn_dict['amount'] = debit * -1

          if not np.isnan(credit):
              txn_dict['amount'] = credit

          balance = None
          balance = str(txns_table[field_list['balance']][ind]).replace(',', '').replace('CR','')
          txn_dict['balance'] = float(balance)
          txn_list.append(txn_dict)

    test_list(txn_list)
    output = file_path + '_output.csv'
    keys = txn_list[0].keys()
    with open(output, 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(txn_list)

bank_txns("111111_Federal_Bank_1.pdf")

validated...


In [8]:
import sys
import re

def empty_search_key(amount):
    category = ''
    if amount < 0:
        category = "Transfer out"
    elif amount >= 0:
        category = "Transfer in"

    return category

def narration_category(narration, amount):
    narration = narration.replace('\\n', '')
    narration = narration.replace('\n', '')
    narration = narration.replace('\\r', '')
    narration = narration.replace('\r', '')
    amount = float(amount)
    category = ''
    search_key = ''
    result = {}
    result['narration'] = narration
    result['amount'] = amount
    result['category'] = ''
    result['search_key'] = ''

    key=narration.strip()
    if narration.startswith("TO ATM"):
        if amount>=0:
          category="Reversal"
        elif amount <=-100:
          category="Cash Withdrawal"
        elif amount >= -99.99 and amount <= 0.01:
          category="Bank Charges"
    elif "LoanDisbursement" in narration:
        if amount>=0:
          category="Personal Loan"
        elif amount<0:
          category="Personal Loan EMI"
    elif "Account Maintenance Charges" in narration:
      category="Bank Charges"
    elif narration.startswith("BY ATM REV") or narration.startswith("BY POS TRANS REV"):
      category="Reversal"
    elif narration.startswith("BY CDM") or narration.startswith("CASH:"):
      category="Cash Deposit"
    elif "Charges for NEFT" in narration:
      category="Cash Deposit"
    elif narration.startswith("CHRG"):
      if ("ATM TXN" in narration or "AWB" in narration or "CASH HANDLING" in narration
          or "DUP CARD" in narration or "ESC RETURN" in narration or "MIN BAL" in narration or
          "NACH MANDATE" in narration or "CHQBOOK" in narration or "MOB ALERT" in narration):
        category="Bank Charges"
    elif narration.startswith("CMS CHARGE-"):
      category="Bank Charges"
    elif narration.startswith("EFT/BYFOM"):
      if amount>=0:
        category="Transfer In"
      elif amount<0:
        category="Transfer Out"
    elif narration.startswith("FedNet Refund"):
      category="Reversal"
    elif "GST" in narration:
      category="Tax"
    elif narration.startswith("IMPS CHARGE"):
      category="Bank Charges"
    elif narration.startswith("IMPS RETURN"):
      category="Bounces I/W payment"
    elif narration.startswith("Loan Rec:"):
      if amount>=0:
        category="Reversal"
      elif amount<0:
        category="Personal Loan EMI"
    elif "DCARD DUP TXN RVSL" in narration:
      category="Reversal"
    elif "MINBAL CHRG RVSL" in narration or "MCPOS REVERSAL" in narration or narration.startswith("RUPAY REFUND"):
      category="Reversal"
    elif narration.startswith("SBINT"):
      category="Interest"
    elif narration.startswith("Unsettled txn"):
      if amount<=0:
        category="Transfer Out"
      if amount>0:
        category="Transfer In"
    elif narration.startswith("VisAppin"):
      if amount<=0:
        category="Transfer Out"
    elif narration.startswith("VISA CR VOUCHER NAT") or narration.startswith("BY ECM TRAN REV"):
      category="Reversal"
    elif narration.startswith("SBINT"):
      category="Interest"

    if category!='':
      result['category']=category.strip()
      result['search_key']=search_key
      return result

    if narration.startswith("ACHCR"):
      val=narration.split("/")
      if len(val)>1:
        key=val[1]
      else:
        key=narration
      remove_list=["BD-","CFD","TP ACH"]
      for i in remove_list:
        if i in key:
          key=key.replace(i,"")
      search_key=key
    elif narration.startswith("BY ACM"):
      key=key.replace("BY ACM","")
      key=re.sub("/\d+","",key)
      key=key.replace("/","")
      key=key.replace("\\","")
      search_key=key
    elif narration.startswith("CLG"):
      val=narration.split("/")
      if len(val)>1:
        key=val[1]
      else:
        key=narration
      search_key=key
    elif narration.startswith("EFT"):
      key=key.replace("EFT/","")
      key=re.sub("RECHARGE","",key)
      key=re.sub("\w+\d+","",key)
      key=re.sub("\d+","",key)
      search_key=key
    elif narration.startswith("FN"):
      if "IMPS/IFO" in narration:
        key=key.replace("FN")
        key=key.replace("IMPS/IFO")
        key=re.sub("/\w{4}\d{7}","",key)
        key=re.sub("/\d+","",key)
        key=key.replace("Pay","")
        val=key.split("/")
        if len(val)>1:
          key=val[0]
        search_key=key
      elif "FN/SHP" in narration:
        key=key.replace("FN/")
        key=key.replace("SHP/")
        key=key.replace("https://")
        key=re.sub("$\d+","",key)
        key=re.sub("\d+/","",key)
        val=key.split("/")
        if len(val)>1:
          key=val[0]
        search_key=key
    elif narration.startswith("FT"):
      if "IFN/ePay/" in narration:
        key=key.replace("FT IFN/ePay/","")
        val=key.split("/")
        if len(val)>1:
          key=val[1]
        search_key=key
      elif "IMPS/IFI/" in narration:
        key=key.replace("FT IMPS/IFI/","")
        val=key.split("/")
        if len(val)>1:
          key=val[1]
        search_key=key
    elif narration.startswith("MB"):
      if "FTB/" in narration:
        val=key.split("/")
        if len(val)>3:
          key=val[3]
          key=re.sub("^Transfer$|^null$|^Personal Exp$|^bills$|^NA$|^family$","",key)
        search_key=key
      elif "IMPS/IFO" in narration:
        val=key.split("/")
        if len(val)>1:
          key=val[len(val)-1]
          key=re.sub("^Transfer$|^null$|^NA$","",key)
        search_key=key
    elif narration.startswith("NFT"):
      val=key.split("/")
      if len(val)>1:
        key=val[1]
      key=re.sub("\w+\d+","",key)
      search_key=key
    elif narration.startswith("POS/"):
      val=key.split("/")
      if len(val)>1:
        key=val[2]
        remove_list=["MSW*","RAZ*","ZES*"]
        for i in remove_list:
          if i  in key:
            key=key.replace(i,"")
      key=key.replace("\\","/")
      key=key.split("/")[0]
      search_key=key
    elif narration.startswith("TFR:FROM") or narration.startswith("TFR:"):
      key=key.replace("TFR:FROM")
      key=key.replace("TFR:")
      search_key=key
    elif narration.startswith("TO ECM"):
      val=key.split("/")
      if len(val)>2:
        key=val[2]
        remove_list=["MSW*","RAZ*","ZES*","TPS*","AIP*"]
        for i in remove_list:
          if i  in key:
            key=key.replace(i,"")
      key=key.replace("\\","/")
      key=key.split("/")[0]
      search_key=key
    elif narration.startswith("UPI"):
      if "IN"  in narration:
        val=key.split("/")
        if len(val)>2:
          val[3]=re.sub('^UPI$|^ UPI$|^NO REMARKS$|^Payment from Phone$|^Payment from Phon$'
          '|^ Payment from P$|^Payment from P$|^Paymant$|^Payman$|^Hello$|^scarch$'
          '|^saler$|^Mobile$|^Phone$|^Retur$|^Cash$|^cash$|^Paid$|^Oid\d+$'
          '^Paytm$|^Self$|^Pay$|^test$|^Test$|^Can$|^Try$|^Emj$|^ic$|^Ok$|^Hi$|^NA$|^N$|^D$|^U$|^M$|^G',"",val[3])
          if val[3]:
            search_key=val[2]+","+val[3]
          else:
            search_key=val[2]
      elif "OUT" in narration:
        val=key.split("/")
        if len(val)>2:
          val[3]=re.sub('^UPI$|^ UPI$|^Payment from Phone$|^Payment from Phon$|'
          '^ Payment from Phon$|^ Payment from Ph$|^Payment from Ph$|'
          '^Payment from P$|^ Payment from P$|^Payment from$|^Payment f$|'
          '^Payment$|^Paymen$|^Self Payment$|^Floating$|^Self P$|^self$|'
          '^Wallet Topup$|Oid\d+$|^amj$|^Amj$|^Fc$|'
          '^NA$|^ NA$|^ok$|^UP$|^Ok$|^aj$|^Sr$|^ic$|^D$|^t$|^U$|^Y$',"",val[3])
          if val[3]:
            search_key=val[2]+","+val[3]
          else:
            search_key=val[2]
    else:
      search_key=narration

    search_key=search_key.strip()
    if search_key=="":
      result['category']=empty_search_key(amount)
      result['search_key']=search_key
    else:
      result['category']=category
      result['search_key']=search_key
    return result

csv_file_name = "111111_Federal_Bank_1.pdf_output.csv"
final_result = []
with open(csv_file_name, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    next(spamreader)
    for row in spamreader:
        result = narration_category(row[1], row[2])
        final_result.append(result)

    output = csv_file_name + '_category.csv'
    keys = ['category', 'search_key', 'narration', 'amount']
    with open(output, 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(final_result)


In [11]:
import re
from datetime import datetime
import io
import sys
import PyPDF4
import pdftotext


def filter_address(line_list):
    address=""
    for i in line_list:
      if "Branch Sol" in i:
        break
      else:
        address+=i+"\n"
    return address

def account_name(line):
    re_match = None
    customer_name = None
    val=line
    if ":" in line:
      val=val.split(":")
      val=val[:len(val)-1]
      val=":".join(val)
    val=re.sub("Branch\\s*Name","",val)
    val=val.strip()
    if re.match("Name\s*?:",val):
      customer_name=re.findall("Name\\s*?:([\w\ ]*)",val)
    if customer_name:
      customer_name=customer_name[0].strip()
    return customer_name

def account_number(line):
    re_match = None
    Acc_no=None
    re_match = re.findall(r'Account\s*Number\s*?:([\d\ ]*)', line)
    if re_match:
      Acc_no=re_match[0].strip()
    return Acc_no

def account_type(line):
    re_match = None
    accType = None
    re_match = re.findall(r'Type\s*of\s*Account\s*?:([\w\ ]*)', line)
    if re_match:
      accType=re_match[0].strip()
    return accType

def statement_for(line):
    re_match = None
    dates={}
    re_match=re.findall(r'period?([\w\-\ ]*)',line)
    if re_match:
      re_match=re_match[0]
      val=re.findall(r'\d{1,2}-\w{1,20}-\d{1,4}', re_match)
    if val:
      dates['start_date']=val[0]
      dates['end_date']=val[1]
    else:
      dates=None
    return dates

def email(line):
    re_match = None
    email_id=None
    re_match=re.findall("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)", line)
    if re_match:
      email_id=re_match[0].strip()
    return email_id

def phone_no(line):
    re_match = None
    mobile_no=None
    re_match=re.findall('Mobile Number\s*?:([\d+\ ]*)',line)
    if re_match:
      mobile_no=re_match[0].strip()
    return mobile_no

def account_data(statement_txt='', alt_statement_txt=''):

    alt_statement_txt = re.sub(' +', ' ', alt_statement_txt)
    alt_statement_txt = alt_statement_txt.split('\n')

    account_data = {'name': '',
                    'address': '',
                    'landline': '',
                    'mobile': '',
                    'email': '',
                    'source_of_data': 'Uploaded Statements',
                    'bank': 'FEDRAL Bank',
                    'account_no': '',
                    'account_type': '',
                     'start_date': '',
                     'end_date': ''}

    for ind,i in enumerate(alt_statement_txt):
      if "Account Number" in i:
        if account_number(i):
          account_data['account_no']=account_number(i)

      if "Name" in i:
        if account_name(i):
          account_data['name']=account_name(i)

      if "Type" in i:
        if account_type(i):
          account_data['account_type']=account_type(i)

      if 'Statement of Account' in i:
        if statement_for(i):
          account_data['start_date']=statement_for(i)['start_date']
          account_data['end_date']=statement_for(i)['end_date']

      if "Mobile Number" in i:
        if phone_no(i):
          account_data['mobile']=phone_no(i)

      if "Email" in i:
        if email(i):
          account_data['email']=email(i)

      if "Communication Address" in i:
        if filter_address(alt_statement_txt[ind:]):
          account_data['address']=filter_address(alt_statement_txt[ind:])
    return account_data

file_name = "/content/111111_Federal_Bank_1.pdf"
password = ''
page_content = ''
alt_page_content = ''
file_content = PyPDF4.PdfFileReader(file_name)
if file_content.isEncrypted:
    try:
        file_content.decrypt(password)
        page1 = file_content.getPage(0)
        page2 = file_content.getPage(1)
        page_content = page1.extractText() + page2.extractText()
    except PyPDF4.utils.PdfReadError:
        print('Incorrect password')
    except:
        page_content = ''
else:
    try:
        page1 = file_content.getPage(0)
        page2 = file_content.getPage(1)
        page_content = page1.extractText() + page2.extractText()
    except:
        page_content = ''
stmnt_pages = []
with open(file_name, "rb") as statement_obj:
    if password:
        stmnt_pages = pdftotext.PDF(statement_obj, password)
    else:
        stmnt_pages = pdftotext.PDF(statement_obj)
    alt_page_content = stmnt_pages[0] + stmnt_pages[1]
customer_details = account_data(page_content, alt_page_content)
print(customer_details)


ModuleNotFoundError: No module named 'pdftotext'